## Train notebook

This is where we train the final model on the whole dataset. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

def get_score(actuals, preds):
    new_test= pd.DataFrame({'Actuals': actuals,'Preds': preds})
    new_test = new_test.loc[new_test['Actuals'] != 0,:]
    return metric(np.array(new_test['Actuals']), np.array(new_test['Preds']))

def mean_encoder(df, col, target = 'Sales'):
    Mean_encoded_subject = no_closed_stores.groupby([col])[target].mean().to_dict() 
    return df[col].map(Mean_encoded_subject)

In [3]:
no_null_sales = pd.read_csv('clean_data.csv', parse_dates = True)
no_null_sales.loc[:,'Date'] = pd.to_datetime(no_null_sales.loc[:,'Date'], format='%Y-%m-%d')
no_closed_stores = no_null_sales[no_null_sales['Open'] == 1]
stores = pd.read_csv('stores_light.csv')

/opt/anaconda3/envs/minicomp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
no_closed_stores['Month'] = no_closed_stores['Date'].dt.month
no_closed_stores['Week'] = no_closed_stores['Date'].dt.week

<ipython-input-4-96a1079c1f72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_closed_stores['Month'] = no_closed_stores['Date'].dt.month
<ipython-input-4-96a1079c1f72>:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  no_closed_stores['Week'] = no_closed_stores['Date'].dt.week
<ipython-input-4-96a1079c1f72>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_closed_stores['Week'] = no_closed_stores['Date'].dt.week


In [5]:
no_closed_stores = no_closed_stores.merge(stores, how='left', on='Store')

In [7]:
rf_set = no_closed_stores.drop(['Date', 'Customers', 'SchoolHoliday', 'Week'], axis = 1)

rf_set.loc[rf_set['Month'] != 12, 'Month'] = 0
rf_set.loc[rf_set['Month'] == 12, 'Month'] = 1
rf_set['Store'] = mean_encoder(rf_set, 'Store')
rf_set['StateHoliday'] = mean_encoder(rf_set, 'StateHoliday')

rf_set['StoreType'] = mean_encoder(rf_set, 'StoreType')
rf_set['Assortment'] = mean_encoder(rf_set, 'Assortment')

rf_set['PromoMonday'] = 0
rf_set.loc[(rf_set['Promo'] == 1) & (rf_set['DayOfWeek'] == 1), 'PromoMonday'] = 1

X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
mask = y > 1200
X = X[mask]
y = y[mask]


X.fillna(0, inplace=True)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42, max_depth = 15, n_estimators = 300, 
                           min_samples_split = 11, max_samples = 0.9, max_features = 7)
rf.fit(X, y)
preds = rf.predict(X)
actuals = np.array(y)
print(f'Score: {get_score(actuals, preds)}')

Score: 15.050982268242436


In [7]:
import pickle

filename = 'model.sav'
pickle.dump(rf, open(filename, 'wb'))